In [ ]:
# import dependencies
from api_keys import census_key
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from census import Census
import hvplot as hv